<a href="https://colab.research.google.com/github/karasu1982/colab_notebook/blob/main/202510_Welfare_Optimized_Recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os, io, zipfile, requests, math, random
import numpy as np
import pandas as pd
from tqdm import tqdm
from collections import defaultdict, Counter
from sklearn.metrics.pairwise import cosine_similarity
import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
# ---------------------------
# 1) Download MovieLens
# ---------------------------
ML_URL = "https://files.grouplens.org/datasets/movielens/ml-latest-small.zip"
ZIP_PATH = "/content/ml-latest-small.zip"
DST_DIR = "/content/ml-latest-small"

if not os.path.exists(DST_DIR):
    print("Downloading MovieLens ...")
    r = requests.get(ML_URL)
    r.raise_for_status()
    with open(ZIP_PATH, "wb") as f:
        f.write(r.content)
    with zipfile.ZipFile(ZIP_PATH) as zf:
        zf.extractall("/content/")
print("Data ready:", os.listdir(DST_DIR))

ratings = pd.read_csv(os.path.join(DST_DIR, "ratings.csv"))
movies  = pd.read_csv(os.path.join(DST_DIR, "movies.csv"))

Data ready: ['README.txt', 'movies.csv', 'links.csv', 'ratings.csv', 'tags.csv']


In [ ]:
# ---------------------------
# 2) Train/Test split (by user, time-aware)
#    - per user: last interaction -> test, rest -> train
# ---------------------------
ratings = ratings.sort_values("timestamp")
def train_test_split_by_user(df):
    train_idx, test_idx = [], []
    for uid, g in df.groupby("userId", sort=False):
        if len(g) == 1:
            train_idx.extend(g.index.tolist())  # no test for singletons
        else:
            test_idx.append(g.index.tolist()[-1])
            train_idx.extend(g.index.tolist()[:-1])
    return df.loc[train_idx].copy(), df.loc[test_idx].copy()

train, test = train_test_split_by_user(ratings)
print(f"Users: {ratings.userId.nunique()}, Items: {ratings.movieId.nunique()}, "
      f"Train: {len(train)}, Test: {len(test)}")

Users: 610, Items: 9724, Train: 100226, Test: 610


In [ ]:
train.head()

,userId,movieId,rating,timestamp
66669,429,165,4.0,828124615
66667,429,161,5.0,828124615
66665,429,150,5.0,828124615
66662,429,22,4.0,828124615
66712,429,432,3.0,828124615


In [ ]:
# ---------------------------
# 3) ID reindex & tensors
# ---------------------------
uids = sorted(ratings.userId.unique())
iids = sorted(ratings.movieId.unique())
uid2idx = {u:i for i,u in enumerate(uids)}
iid2idx = {i:j for j,i in enumerate(iids)}
idx2uid = {i:u for u,i in uid2idx.items()}
idx2iid = {j:i for i,j in iid2idx.items()}

train["u_idx"] = train["userId"].map(uid2idx)
train["i_idx"] = train["movieId"].map(iid2idx)
test["u_idx"]  = test["userId"].map(uid2idx)
test["i_idx"]  = test["movieId"].map(iid2idx)

n_users = len(uids); n_items = len(iids)
r_min, r_max = 0.5, 5.0

In [ ]:
# ---------------------------
# 4) PyTorch MF model
# ---------------------------

class MF(nn.Module):
    def __init__(self, n_users, n_items, n_factors=64):
        super().__init__()
        self.P = nn.Embedding(n_users, n_factors)
        self.Q = nn.Embedding(n_items, n_factors)
        self.bu = nn.Embedding(n_users, 1)
        self.bi = nn.Embedding(n_items, 1)
        self.mu = nn.Parameter(torch.tensor(3.5))  # global mean
        nn.init.normal_(self.P.weight, std=0.05)
        nn.init.normal_(self.Q.weight, std=0.05)
        nn.init.zeros_(self.bu.weight)
        nn.init.zeros_(self.bi.weight)

    def forward(self, u_idx, i_idx):
        pu = self.P(u_idx)
        qi = self.Q(i_idx)
        dot = (pu * qi).sum(dim=1, keepdim=True)
        pred = self.mu + self.bu(u_idx) + self.bi(i_idx) + dot
        return pred.squeeze(1)

model = MF(n_users, n_items, n_factors=64).to(DEVICE)
opt = optim.Adam(model.parameters(), lr=5e-3, weight_decay=1e-5)
loss_fn = nn.MSELoss()

# DataLoader（シンプルに全部載せでもOK）
u_t = torch.tensor(train["u_idx"].values, dtype=torch.long, device=DEVICE)
i_t = torch.tensor(train["i_idx"].values, dtype=torch.long, device=DEVICE)
y_t = torch.tensor(train["rating"].values, dtype=torch.float32, device=DEVICE)


In [ ]:
# ---------------------------
# 5) Train
# ---------------------------
EPOCHS = 15
bs = 8192
n = len(train)
for ep in range(1, EPOCHS+1):
    model.train()
    perm = torch.randperm(n, device=DEVICE)
    epoch_loss = 0.0
    for st in range(0, n, bs):
        idx = perm[st:st+bs]
        u_b, i_b, y_b = u_t[idx], i_t[idx], y_t[idx]
        opt.zero_grad(set_to_none=True)
        y_hat = model(u_b, i_b)
        loss = loss_fn(y_hat, y_b)
        loss.backward()
        opt.step()
        epoch_loss += loss.item() * len(idx)
    print(f"Epoch {ep:02d} | train MSE: {epoch_loss/n:.4f}")


Epoch 01 | train MSE: 1.0583
Epoch 02 | train MSE: 0.9513
Epoch 03 | train MSE: 0.8272
Epoch 04 | train MSE: 0.6690
Epoch 05 | train MSE: 0.5153
Epoch 06 | train MSE: 0.4002
Epoch 07 | train MSE: 0.3156
Epoch 08 | train MSE: 0.2538
Epoch 09 | train MSE: 0.2080
Epoch 10 | train MSE: 0.1731
Epoch 11 | train MSE: 0.1464
Epoch 12 | train MSE: 0.1256
Epoch 13 | train MSE: 0.1092
Epoch 14 | train MSE: 0.0960
Epoch 15 | train MSE: 0.0850


In [ ]:
# ---------------------------
# 6) Candidate scoring
# ---------------------------
ALL_USERS = uids
ALL_ITEMS = iids

train_ui = set(zip(train.userId, train.movieId))
user_to_seen = defaultdict(set)
for u, i in train_ui:
    user_to_seen[u].add(i)

TOPN_CANDIDATES = 200  # per-user candidate pool
model.eval()
with torch.no_grad():
    P = model.P.weight.detach().cpu().numpy()
    Q = model.Q.weight.detach().cpu().numpy()
    bu = model.bu.weight.detach().cpu().numpy().squeeze()
    bi = model.bi.weight.detach().cpu().numpy().squeeze()
    mu = float(model.mu.detach().cpu())

user_item_scores = {}
for u in tqdm(ALL_USERS, desc="Scoring"):
    u_idx = uid2idx[u]
    # score all items vectorized
    scores = mu + bu[u_idx] + bi + (P[u_idx] @ Q.T)
    # mask seen
    mask = np.ones(n_items, dtype=bool)
    if len(user_to_seen[u]) > 0:
        seen_idx = [iid2idx[i] for i in user_to_seen[u]]
        mask[seen_idx] = False  # we'll filter after mapping
    pairs = []
    for j in range(n_items):
        item_raw = idx2iid[j]
        if item_raw in user_to_seen[u]:
            continue
        pairs.append((item_raw, float(scores[j])))
    pairs.sort(key=lambda x: x[1], reverse=True)
    user_item_scores[u] = pairs[:TOPN_CANDIDATES]

# For ILDI: item factors (Q) in raw id space
item_factors = { idx2iid[j]: Q[j] for j in range(n_items) }


Scoring: 100%|██████████| 610/610 [00:11<00:00, 53.78it/s]


In [ ]:
# ---------------------------
# 7) Baselines (K=10)
# ---------------------------
K = 10
topk_baseline = {u: [i for i,_ in user_item_scores[u][:K]] for u in ALL_USERS}

pop_counts = Counter(train.movieId.tolist())
global_pop = [i for i,_c in sorted(pop_counts.items(), key=lambda x: x[1], reverse=True)]
pop_baseline = {}
for u in ALL_USERS:
    pop_list = [i for i in global_pop if i not in user_to_seen[u]][:K]
    pop_baseline[u] = pop_list

In [ ]:
# ---------------------------
# 8) Welfare-aware reranking (assignment)
# ---------------------------
alpha = 0.7
EXPO_CAP = 100
CENTER = 3.5

def f_utility(score, center=CENTER):
    x = max(0.0, score - center)
    return math.log1p(math.exp(x))  # softplus

# Precompute utilities
user_candidates = {}
for u, lst in user_item_scores.items():
    user_candidates[u] = [(i, s, f_utility(s)) for (i, s) in lst if s > 0]

assigned = {u: [] for u in ALL_USERS}
per_user_utility = defaultdict(float)
item_exposure = Counter()

print("Running welfare-aware greedy assignment...")
rng = np.random.default_rng(SEED)
user_order = list(ALL_USERS)
for round_idx in range(K):
    rng.shuffle(user_order)
    for u in user_order:
        if len(assigned[u]) >= K:
            continue
        best_i, best_gain = None, -1e9
        base_util = per_user_utility[u]
        for (i, score, util) in user_candidates[u]:
            if i in user_to_seen[u]: continue
            if item_exposure[i] >= EXPO_CAP: continue
            gain_nash = math.log((base_util + util + 1e-9)) - math.log((base_util + 1e-9))
            gain_util = util
            gain = alpha * gain_nash + (1 - alpha) * gain_util
            if gain > best_gain:
                best_gain = gain
                best_i = (i, util)
        if best_i is not None:
            i, util = best_i
            assigned[u].append(i)
            per_user_utility[u] += util
            item_exposure[i] += 1

welfare_rec = assigned

Running welfare-aware greedy assignment...


In [ ]:
# ---------------------------
# 9) Metrics
# ---------------------------


#指標名	目的
#① Utilitarian Welfare（総効用）	全ユーザーの満足度の合計を測る
#② Nash Welfare（ナッシュ厚生）	公平性を考慮した全体効用を測る
#③ Gini Coefficient（ジニ係数）	ユーザー間の不平等度を測る
#④ Coverage（被推薦率）	推薦がどれだけ多くのアイテムをカバーしているか
#⑤ ILDI（Intra-List Diversity）	各ユーザーのリスト内でどれだけ多様なアイテムを提示できているか

def pred_score(u_raw, i_raw):
    u = uid2idx[u_raw]; j = iid2idx[i_raw]
    return mu + bu[u] + bi[j] + (P[u] @ Q[j])

def utilitarian_welfare(assignments, use_transform=True):
    total = 0.0
    for u, items in assignments.items():
        for i in items:
            s = pred_score(u, i)
            total += f_utility(s) if use_transform else s
    return total

def nash_welfare(assignments):
    eps = 1e-9
    total = 0.0
    for u, items in assignments.items():
        u_util = 0.0
        for i in items:
            s = pred_score(u, i)
            u_util += f_utility(s)
        total += math.log(u_util + eps)
    return total

def gini(values):
    arr = np.array(values, dtype=float)
    if np.amin(arr) < 0:
        arr -= np.amin(arr)
    mean = np.mean(arr)
    if mean == 0: return 0.0
    diff_sum = np.sum(np.abs(arr.reshape(-1,1) - arr.reshape(1,-1)))
    return diff_sum / (2 * len(arr) * np.sum(arr))

def per_user_utility_list(assignments):
    res = []
    for u, items in assignments.items():
        s = sum(f_utility(pred_score(u, i)) for i in items)
        res.append(s)
    return res

def coverage(assignments):
    items = set()
    for u, lst in assignments.items():
        items.update(lst)
    return len(items) / len(ALL_ITEMS)

def ildi(assignments):
    ild_vals = []
    for u, lst in assignments.items():
        vecs = [item_factors[i] for i in lst if i in item_factors]
        if len(vecs) <= 1:
            continue
        M = cosine_similarity(vecs)
        n = len(vecs)
        sims = []
        for a in range(n):
            for b in range(a+1, n):
                sims.append(1.0 - M[a,b])
        if sims:
            ild_vals.append(float(np.mean(sims)))
    return float(np.mean(ild_vals)) if ild_vals else np.nan

def evaluate(assignments, name):
    util_w = utilitarian_welfare(assignments)
    nash_w = nash_welfare(assignments)
    g = gini(per_user_utility_list(assignments))
    cov = coverage(assignments)
    d = ildi(assignments)
    print(f"[{name}] Utilitarian(sum f): {util_w:,.2f} | Nash(sum log): {nash_w:,.2f} | "
          f"Gini(↓ good): {g:.4f} | Coverage: {cov*100:.2f}% | ILDI: {d:.4f}")

print("\n=== Evaluation ===")
evaluate(topk_baseline, "Per-User TopK (MF)")
evaluate(pop_baseline,  "Popularity")
evaluate(welfare_rec,   f"Welfare-aware (alpha={alpha:.1f}, cap={EXPO_CAP})")


=== Evaluation ===
[Per-User TopK (MF)] Utilitarian(sum f): 9,378.19 | Nash(sum log): 1,658.95 | Gini(↓ good): 0.0900 | Coverage: 11.07% | ILDI: 0.8019
[Popularity] Utilitarian(sum f): 6,068.81 | Nash(sum log): 1,390.06 | Gini(↓ good): 0.1105 | Coverage: 1.24% | ILDI: 0.7882
[Welfare-aware (alpha=0.7, cap=100)] Utilitarian(sum f): 10,708.18 | Nash(sum log): 1,737.54 | Gini(↓ good): 0.1015 | Coverage: 3.21% | ILDI: 0.0097


## 📊 評価結果

| モデル                                  | Utilitarian（総効用） | Nash（公平性を考慮）    | Gini（↓良）     | Coverage     | ILDI（多様性）    |
| ------------------------------------ | ---------------- | --------------- | ------------ | ------------ | ------------ |
| **① Per-User TopK (MF)**             | **9,378.19**     | 1,658.95        | ✅ **0.0900** | ✅ **11.07%** | ✅ **0.8019** |
| **② Popularity**                     | 6,068.81         | 1,390.06        | 0.1105       | ❌ **1.24%**  | 0.7882       |
| **③ Welfare-aware (α=0.7, cap=100)** | 🟢 **10,708.18** | 🟢 **1,737.54** | 0.1015       | ⚠️ **3.21%** | ❌ **0.0097** |


In [ ]:
# ============================================
# 8') Welfare-aware reranking with diversity penalty
#    - 既存の貪欲割当を差し替え
#    - 目的: α*ΔNash + (1-α)*ΔUtil  − λ_div * (スレート内平均コサイン類似度)
# ============================================

# 推薦スレート内の“平均コサイン類似度”をペナルティとして使う
def slate_similarity_penalty(user_items, candidate_i, item_factors):
    if not user_items:
        return 0.0
    v = item_factors.get(candidate_i)
    if v is None:
        return 0.0
    v_norm = np.linalg.norm(v) + 1e-9
    sims = []
    for j in user_items:
        w = item_factors.get(j)
        if w is None:
            continue
        s = float(np.dot(v, w) / ((np.linalg.norm(w) + 1e-9) * v_norm))
        sims.append(s)
    return float(np.mean(sims)) if sims else 0.0

def welfare_rerank_diverse(
    alpha=0.7,
    expo_cap=30,          # ★ 多様性確保のため露出上限を強めに下げる（例: 100→30）
    K=10,
    lambda_div=0.2        # ★ 多様性重み（0.1〜0.5あたりをチューニング）
):
    assigned = {u: [] for u in ALL_USERS}
    per_user = defaultdict(float)
    item_exp = Counter()
    rng = np.random.default_rng(SEED)
    user_order = list(ALL_USERS)

    for r in range(K):
        rng.shuffle(user_order)
        for u in user_order:
            if len(assigned[u]) >= K:
                continue
            best_i, best_gain = None, -1e9
            base_u = per_user[u]
            # そのユーザーの候補から最も増分利益が大きいアイテムを選ぶ
            for (i, score, util0) in user_candidates[u]:
                if i in user_to_seen[u]:
                    continue
                if item_exp[i] >= expo_cap:
                    continue
                util = util0  # f_utility(score) を事前計算済み
                # Nashの限界増分
                gain_nash = math.log((base_u + util + 1e-9)) - math.log((base_u + 1e-9))
                gain_util = util
                # スレート類似ペナルティ（平均コサイン類似度）
                pen = slate_similarity_penalty(assigned[u], i, item_factors)
                gain = alpha * gain_nash + (1 - alpha) * gain_util - lambda_div * pen
                if gain > best_gain:
                    best_gain = gain
                    best_i = (i, util)
            if best_i is not None:
                i, util = best_i
                assigned[u].append(i)
                per_user[u] += util
                item_exp[i] += 1
    return assigned

# 実行：多様性付き再ランキング
alpha_new = 0.7
EXPO_CAP_NEW = 30
LAMBDA_DIV = 0.2

welfare_rec_div = welfare_rerank_diverse(
    alpha=alpha_new, expo_cap=EXPO_CAP_NEW, K=K, lambda_div=LAMBDA_DIV
)

print("=== Evaluation (diversity-aware) ===")
evaluate(topk_baseline, "Per-User TopK (MF)")
evaluate(pop_baseline,  "Popularity")
evaluate(welfare_rec,   f"Welfare-aware (old, alpha={alpha:.1f}, cap={EXPO_CAP})")
evaluate(welfare_rec_div, f"Welfare-aware + Diversity (alpha={alpha_new:.1f}, cap={EXPO_CAP_NEW}, λ_div={LAMBDA_DIV})")

# サンプル表示
show_sample(u=1, n=10, title="User #1 (diversity-aware)")
print("Welfare(div):", movies[movies.movieId.isin(welfare_rec_div[1])]["title"].tolist())


=== Evaluation (diversity-aware) ===
[Per-User TopK (MF)] Utilitarian(sum f): 9,378.19 | Nash(sum log): 1,658.95 | Gini(↓ good): 0.0900 | Coverage: 11.07% | ILDI: 0.8019
[Popularity] Utilitarian(sum f): 6,068.81 | Nash(sum log): 1,390.06 | Gini(↓ good): 0.1105 | Coverage: 1.24% | ILDI: 0.7882
[Welfare-aware (old, alpha=0.7, cap=100)] Utilitarian(sum f): 10,708.18 | Nash(sum log): 1,737.54 | Gini(↓ good): 0.1015 | Coverage: 3.21% | ILDI: 0.0097
[Welfare-aware + Diversity (alpha=0.7, cap=30, λ_div=0.2)] Utilitarian(sum f): 10,189.56 | Nash(sum log): 1,706.94 | Gini(↓ good): 0.1035 | Coverage: 7.77% | ILDI: 0.6470

== User #1 (diversity-aware) (userId=1) ==
TopK: ['City of Lost Children, The (Cité des enfants perdus, La) (1995)', 'Kung Fu Hustle (Gong fu) (2004)', 'Madness of King George, The (1994)', 'Player, The (1992)', 'Lord of the Rings: The Return of the King, The (2003)', "Bill & Ted's Excellent Adventure (1989)", 'There Will Be Blood (2007)', 'District 9 (2009)', 'Kiss Kiss Bang B

## 📊 評価結果（再掲）

| モデル                                         | Utilitarian（総効用） | Nash（公平性）       | Gini（↓良）     | Coverage     | ILDI（多様性）     |
| ------------------------------------------- | ---------------- | --------------- | ------------ | ------------ | ------------- |
| **① Per-User TopK (MF)**                    | 9,378.19         | 1,658.95        | ✅ **0.0900** | ✅ **11.07%** | ✅ **0.8019**  |
| **② Popularity**                            | 6,068.81         | 1,390.06        | ❌ 0.1105     | ❌ **1.24%**  | 0.7882        |
| **③ Welfare-aware (old)**                   | 🟢 **10,708.18** | 🟢 **1,737.54** | 0.1015       | ❌ **3.21%**  | ❌ **0.0097**  |
| **④ Welfare-aware + Diversity (λ_div=0.2)** | **10,189.56**    | **1,706.94**    | 0.1035       | 🟢 **7.77%** | 🟢 **0.6470** |

In [ ]:
# ---------------------------
# 10) Show sample user
# ---------------------------
def show_sample(u=1, n=10, title="Sample User"):
    def to_titles(item_ids):
        return movies[movies.movieId.isin(item_ids)][["movieId","title"]].merge(
            pd.DataFrame({"movieId":item_ids}), on="movieId", how="right")["title"].tolist()
    print(f"\n== {title} (userId={u}) ==")
    print("TopK:", to_titles(topk_baseline[u][:n]))
    print("Welfare:", to_titles(welfare_rec[u][:n]))
    print("Popularity:", to_titles(pop_baseline[u][:n]))

show_sample(u=1, n=10, title="User #1")


== User #1 (userId=1) ==
TopK: ['City of Lost Children, The (Cité des enfants perdus, La) (1995)', 'Kung Fu Hustle (Gong fu) (2004)', 'Madness of King George, The (1994)', 'Player, The (1992)', 'Lord of the Rings: The Return of the King, The (2003)', "Bill & Ted's Excellent Adventure (1989)", 'There Will Be Blood (2007)', 'District 9 (2009)', 'Kiss Kiss Bang Bang (2005)', 'Mystery Science Theater 3000: The Movie (1996)']
Welfare: ['City of Lost Children, The (Cité des enfants perdus, La) (1995)', 'City of Lost Children, The (Cité des enfants perdus, La) (1995)', 'City of Lost Children, The (Cité des enfants perdus, La) (1995)', 'City of Lost Children, The (Cité des enfants perdus, La) (1995)', 'City of Lost Children, The (Cité des enfants perdus, La) (1995)', 'City of Lost Children, The (Cité des enfants perdus, La) (1995)', 'City of Lost Children, The (Cité des enfants perdus, La) (1995)', 'City of Lost Children, The (Cité des enfants perdus, La) (1995)', 'City of Lost Children, The 